In [1]:
import pandas as pd
import glob

pd.set_option("display.precision", 2) #always just show 2 decimal points


#Read in all country data
path = r'/Users/rvpazos/Documents/mvtec/ManifestoProject/workshop2-manifesto-rebeccaFolder/country-data' # use your path
all_files = glob.glob(path + "/*.csv")

all_files
li = []
for filename in all_files:
    df = pd.read_csv(filename,header=0)
    li.append(df)

allData = pd.concat(li, axis=0)

# allData.reset_index()
# allData = allData.set_index(pd.DatetimeIndex(allData['index'])).drop(['index'], axis=1)

allData = allData.rename(columns={'index':'yearObject'})

allData



,yearObject,parfam,countryname,environ,culture,equality,pervote,rile,electionDate,partyname
0,1989-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left
1,1990-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left
2,1991-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left
3,1992-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left
4,1993-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left
...,...,...,...,...,...,...,...,...,...,...
228,2016-01-01,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists'
229,2017-01-01,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists'
230,2018-01-01,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists'
231,2019-01-01,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists'


In [2]:
#Parfam renamed
import numpy as np 

#Change parfam codes to abbreviations for better identification
conditions = [
    (allData['parfam'] == 10),
    (allData['parfam'] == 20),
    (allData['parfam'] == 30),
    (allData['parfam'] == 40),
    (allData['parfam'] == 50),
    (allData['parfam'] == 60),
    (allData['parfam'] == 70),
    (allData['parfam'] == 80),
    (allData['parfam'] == 90),
    (allData['parfam'] == 95),
    (allData['parfam'] == 98),
]

values = ['ECO','LEF','SOC','LIB','CHR', 'CON', 'NAT', 'AGR', 'ETH', 'SIP', 'DIV']

allData['parfamName'] = np.select(conditions,values)


allData.head()


,yearObject,parfam,countryname,environ,culture,equality,pervote,rile,electionDate,partyname,parfamName
0,1989-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO
1,1990-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO
2,1991-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO
3,1992-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO
4,1993-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO


In [3]:
#Extract year

allData['yearObject'] = pd.to_datetime(allData['yearObject'], format='%Y-%m-%d')
allData['yearIndex'] = allData['yearObject'].dt.year

allData.drop('yearObject', axis='columns', inplace = True)


In [4]:
#Add column with static rile instead of dynamic just in case

conditionsStatic = [
    (allData['parfam'] == 10),
    (allData['parfam'] == 20),
    (allData['parfam'] == 30),
    (allData['parfam'] == 40),
    (allData['parfam'] == 50),
    (allData['parfam'] == 60),
    (allData['parfam'] == 70),
    (allData['parfam'] == 80),
    (allData['parfam'] == 90),
    (allData['parfam'] == 95),
    (allData['parfam'] == 98),
]

valuesStatic = [-13.65,-23.66,-14,4.61,6.13,11.64,10.25,1.01,-9.04,3.36,-10.52]

allData['staticRile'] = np.select(conditionsStatic,valuesStatic)

allData


,parfam,countryname,environ,culture,equality,pervote,rile,electionDate,partyname,parfamName,yearIndex,staticRile
0,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO,1989,-13.65
1,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO,1990,-13.65
2,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO,1991,-13.65
3,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO,1992,-13.65
4,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO,1993,-13.65
...,...,...,...,...,...,...,...,...,...,...,...,...
228,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists',SIP,2016,3.36
229,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists',SIP,2017,3.36
230,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists',SIP,2018,3.36
231,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists',SIP,2019,3.36


In [5]:

allData = allData[['yearIndex','parfamName','parfam','countryname','environ','culture','equality','pervote','rile','staticRile','electionDate','partyname']]
df2 = allData.set_index('parfamName')
df2.to_csv('allCountries.csv')

df2

,yearIndex,parfam,countryname,environ,culture,equality,pervote,rile,staticRile,electionDate,partyname
parfamName,,,,,,,,,,,
ECO,1989,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,-13.65,1989-09-01,Green Left
ECO,1990,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,-13.65,1989-09-01,Green Left
ECO,1991,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,-13.65,1989-09-01,Green Left
ECO,1992,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,-13.65,1989-09-01,Green Left
ECO,1993,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,-13.65,1989-09-01,Green Left
...,...,...,...,...,...,...,...,...,...,...,...
SIP,2016,95,Estonia,0.0,0.0,0.0,7.12,62.5,3.36,1992-09-01,Electoral Union ‘Independent Royalists'
SIP,2017,95,Estonia,0.0,0.0,0.0,7.12,62.5,3.36,1992-09-01,Electoral Union ‘Independent Royalists'
SIP,2018,95,Estonia,0.0,0.0,0.0,7.12,62.5,3.36,1992-09-01,Electoral Union ‘Independent Royalists'


In [6]:
#create grouped file for CSV

import numpy as np

pt = pd.pivot_table(df2, index={'parfamName','yearIndex'}, values=['parfam','environ','culture','equality','pervote','rile','staticRile'], aggfunc={'parfam' : 'count', 'environ' : 'sum','culture' : 'sum','equality' : 'sum', 'pervote' : 'sum', 'rile' : 'mean', 'staticRile':'min'}, dropna=True)
pt = pt.rename(columns={'parfam':'parfamCount'})
pt['avgVote'] = (pt['pervote'])/(pt['parfamCount'])

pt

culture  environ  equality  parfamCount  pervote   rile  \
parfamName yearIndex                                                            
AGR        1944          0.00     0.00      0.00            1    13.60  23.81   
           1945          1.10     0.00      4.40            2    34.95  13.55   
           1946          1.10     0.00      4.40            2    34.95  13.55   
           1947          1.10     0.00      4.40            2    34.95  13.55   
           1948          2.33     0.00     12.56            3    42.20  -4.63   
...                       ...      ...       ...          ...      ...    ...   
SOC        2016         83.14    88.64    226.87           29   612.85 -16.98   
           2017         77.14    88.70    229.24           27   565.30 -15.31   
           2018         85.92   102.60    193.29           27   548.48 -15.11   
           2019         81.18   126.37    213.72           27   561.70 -17.59   
           2020         79.06   123.29    204.29           26   533.44 -17.16   

                      staticRile  avgVote  
parfamName yearIndex                       
AGR        1944             1.01    13.60  
           1945             1.01    17.48  
           1946             1.01    17.48  
           1947             1.01    17.48  
           1948             1.01    14.07  
...                          ...      ...  
SOC        2016           -14.00    21.13  
           2017           -14.00    20.94  
           2018           -14.00    20.31  
           2019           -14.00    20.80  
           2020           -14.00    20.52  

[749 rows x 8 columns]

In [7]:
pt.to_csv('grouped-data.csv')

In [22]:
import json

# pt_json = pt.reset_index()
# pt_json = pt_json.set_index({'parfamName','yearIndex'})
# pt_json

result = pt.to_json('test2.json',orient='index')




In [8]:
spainDF = allData.loc[(allData['countryname']=='Spain')] 

decadesFilter = ['1990-01-01','1995-01-01','2000-01-01','2005-01-01','2010-01-01','2015-01-01','2020-01-01']
filteredSpain = spainDF[spainDF.index.isin(decadesFilter)]
filteredSpain


ptSpain = pd.pivot_table(filteredSpain, index=['index','parfamName'], values=['parfam','per503','pervote','environWeighted'], aggfunc={'parfam' : 'count', 'per503' : 'sum', 'pervote' : 'sum', 'environWeighted' : 'sum'}, dropna=True)
ptSpain['avgVote'] = (ptSpain['pervote'])/(ptSpain['parfam'])

ptSpain.to_csv('spain-grouped.csv')

In [95]:
n_by_fam = allData.groupby(['index','parfam'])['parfam'].count()
n_by_fam


index       parfam
1944-01-01  20         1
            30         1
            40         1
            60         1
            80         1
                      ..
2020-01-01  70        24
            80        12
            90        13
            95        16
            98         4
Name: parfam, Length: 750, dtype: int64